In [ ]:
import pandas as pd

# Load the data from the uploaded CSV file
file_path = r"C:\Users\verma\Downloads\problems\problems\frag0.csv"
data = pd.read_csv(file_path)

In [ ]:
import matplotlib.pyplot as plt

# Extract the coordinates
x = data.iloc[:, 2]
y = data.iloc[:, 3]

# Create a scatter plot of the points
plt.figure(figsize=(10, 8))
plt.scatter(x, y, s=10, color='blue')
plt.xlabel('X Coordinates')
plt.ylabel('Y Coordinates')
plt.grid(True)

# Remove the coordinate axes
plt.axis('off')

# Save the plot as an image file
plt.savefig('scatter_plot.png', bbox_inches='tight', pad_inches=0)

# Show the plot
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def straighten_contour(contour, epsilon=0.02):
    # Approximate the contour to reduce the number of points
    approx = cv2.approxPolyDP(contour, epsilon * cv2.arcLength(contour, True), True)
    return approx

def detect_and_straighten_shapes(image_path, epsilon=0.02):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Binarize the image
    _, bin_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(bin_img.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create images for overdrawn and untouched contours
    overdrawn_img = np.full_like(image, 255)  # Start with a white background
    untouched_img = image.copy()  # Start with the original image

    rng = np.random.default_rng(123)

    # Iterate over contours and process shapes
    for contour in contours:
        # Compute convex hull
        hull = cv2.convexHull(contour)

        # Compute circularity
        area = cv2.contourArea(hull)
        perimeter = cv2.arcLength(hull, True)

        # Check if perimeter is zero to avoid division by zero error
        if perimeter == 0:
            continue

        circularity = (4 * np.pi * area) / (perimeter * perimeter)

        if circularity > 0.95:
            # CIRCLE: Find min enclosing circle
            center, radius = cv2.minEnclosingCircle(contour)
            center = tuple(map(int, center))
            radius = int(radius)

            # Draw the perfect circle on overdrawn image
            color = tuple(rng.integers(0, 256, size=3).tolist())
            cv2.circle(overdrawn_img, center, radius, color, 5)

            # Remove original circle from untouched image
            cv2.drawContours(untouched_img, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
        else:
            # STRAIGHTEN OTHER SHAPES
            straightened_contour = straighten_contour(contour, epsilon)

            # Draw the straightened contour on overdrawn image
            cv2.drawContours(overdrawn_img, [straightened_contour], -1, (0, 255, 0), 4)

            # Remove original shape from untouched image
            cv2.drawContours(untouched_img, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

    # Convert BGR images to RGB for displaying with matplotlib
    overdrawn_rgb = cv2.cvtColor(overdrawn_img, cv2.COLOR_BGR2RGB)
    untouched_rgb = cv2.cvtColor(untouched_img, cv2.COLOR_BGR2RGB)

    # Display the images
    plt.figure(figsize=(20, 10))
    
    # Display overdrawn contours
    plt.subplot(1, 2, 1)
    plt.imshow(overdrawn_rgb)
    plt.title("Overdrawn Contours")
    plt.axis('off')
    plt.show()

input_image_path = r"C:\Users\verma\OneDrive\Desktop\scatter_plot.png"
detect_and_straighten_shapes(input_image_path, epsilon=0.01)
